## Clean Boxscore Data & Merge with Season Data

In [1]:
import pandas as pd
import os
import pickle
import pipeline as p
import datetime as dt

# from collections import defaultdict

%load_ext autoreload
%autoreload 2

pd.set_eng_float_format(accuracy=1, use_eng_prefix=True)

In [ ]:
url_team_dict = p.open_pkl('TeamDicts/url_team_dict.pkl') 

master_df = p.open_pkl('Data/master_df.pkl')
# master_df = p.open_pkl('Data/season_stats_df.pkl')
# master_df = p.open_pkl('Data/season_stats_df_2018.pkl')

dir_name = 'Data/BxscHtmls/'
# os.chdir('BxscHtmls/')

In [ ]:
# For all files in directory
bxsc_df = pd.DataFrame()

for f in os.listdir(dir_name):
    count = 0
    bxsc_html = p.open_pkl(dir_name + f)
    year = int(f[-8:-4])
    print(year)
    for key, val in bxsc_html.items():
        week_date = dt.datetime.strptime(key.split("/")[2][:8], '%Y%m%d')
        team = url_team_dict[key[-7:-4]][2]
        week = master_df[(master_df['Date'] == week_date) & (master_df['Team'] == team)]['Week'].values[0]

        df = p.clean_bxsc_data(val, week, year)

        # Append boxscore df to bxsc_df
        bxsc_df = bxsc_df.append(df)

        count += 1
        if count%20 == 0:
            print(count)
    
print ('Done!')

p.pkl_this('../Data/bxsc_df.pkl', bxsc_df)

In [ ]:
# for a single year. Must specify filename

f = 'bxsc_html_2018.pkl'

count = 0
bxsc_html = p.open_pkl(dir_name + f)
year = int(f[-8:-4])
# print(year)

bxsc_df = pd.DataFrame()

for key, val in bxsc_html.items():
    week_date = dt.datetime.strptime(key.split("/")[2][:8], '%Y%m%d')

    if week_date < dt.datetime.now():
        print(week_date)
        team = url_team_dict[key[-7:-4]][2]
        print(team)
        week = master_df[(master_df['Date'] == week_date) & (master_df['Team'] == team)]['Week'].values[0]
        try:
            df = p.clean_bxsc_data(val, week, year)

            # Append boxscore df to bxsc_df
            bxsc_df = bxsc_df.append(df)

            count += 1
            if count%20 == 0:
                print(count)
        except:
            print('Error')
    

print ('Done!')

p.pkl_this(f'Data/bxsc_df_{str(year)}.pkl', bxsc_df)

In [ ]:
# Merge
df = pd.merge(master_df, bxsc_df, how='left', on=['Team', 'Week','Year'])

#### Check

In [ ]:
df.columns

In [ ]:
len(master_df), len(df)

In [ ]:
df.rename(columns={'1stD-O':'1stD', 'TO-D':'DefTO'}, inplace=True)

### Save

In [ ]:
# p.pkl_this('../merged_df_2018.pkl', df)
p.pkl_this('../merged_df.pkl', df)